In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [47]:
farms = pd.read_csv("../data-mining/farms.csv")
rubrics = pd.read_csv("../data-mining/rubrics.csv")
stocks = pd.read_csv("../data-mining/stocks.csv")

In [48]:
def departement_of(code_postal: int) -> int:
    departement = int(str(code_postal)[:2])

    # dom_tom departement code is 3 digits-long instead of 2
    dom_tom: bool = departement == 97
    if dom_tom:
        departement = int(str(code_postal)[:3])

    return departement

farms["departement"] = farms.codePostal.apply(departement_of)

In [49]:
farms_stocks = farms.merge(stocks, left_on="name", right_on="raison_sociale")

In [50]:
farms_stocks = farms_stocks[["name", "rubric_id", "quantite", "unite", "departement", "bovins", "porcs", "volailles"]]
farms_stocks

,name,rubric_id,quantite,unite,departement,bovins,porcs,volailles
0,GAEC DES BOURRYS,2102,1069.0,Animaux-Eq,89,False,True,False
1,GAEC PORC D'ALGRES,3660,860.0,NaN,81,False,True,False
2,GAEC PORC D'ALGRES,2102,860.0,NaN,81,False,True,False
3,GAEC FERME H ET L,2101,105.0,NaN,50,True,False,False
4,GAEC FERME H ET L,2101,150.0,NaN,50,True,False,False
...,...,...,...,...,...,...,...,...
23682,DE LA CLINERIE (GAEC) LEFEVRE Christian,2111,25500.0,Animaux-Eq,45,False,True,True
23683,DE LA CLINERIE (GAEC) LEFEVRE Christian,2111,25500.0,NaN,45,False,True,True
23684,DES JOURNETS (SCEA),2102,1857.0,Animaux-Eq,41,False,True,False
23685,EARL DES MUSSETS,2111,40000.0,u,41,False,False,True


In [51]:
def animal_type(rubric_id: str) -> str:
    if rubric_id == "2101":
        return "BOVINS"

    if rubric_id == "2102" or rubric_id == "3660":
        return "PORCS"

    if rubric_id == "2111":
        return "VOLAILLES"

farms_stocks["type"] = farms_stocks.rubric_id.apply(animal_type)

In [52]:
farms_stocks

,name,rubric_id,quantite,unite,departement,bovins,porcs,volailles,type
0,GAEC DES BOURRYS,2102,1069.0,Animaux-Eq,89,False,True,False,PORCS
1,GAEC PORC D'ALGRES,3660,860.0,NaN,81,False,True,False,PORCS
2,GAEC PORC D'ALGRES,2102,860.0,NaN,81,False,True,False,PORCS
3,GAEC FERME H ET L,2101,105.0,NaN,50,True,False,False,BOVINS
4,GAEC FERME H ET L,2101,150.0,NaN,50,True,False,False,BOVINS
...,...,...,...,...,...,...,...,...,...
23682,DE LA CLINERIE (GAEC) LEFEVRE Christian,2111,25500.0,Animaux-Eq,45,False,True,True,VOLAILLES
23683,DE LA CLINERIE (GAEC) LEFEVRE Christian,2111,25500.0,NaN,45,False,True,True,VOLAILLES
23684,DES JOURNETS (SCEA),2102,1857.0,Animaux-Eq,41,False,True,False,PORCS
23685,EARL DES MUSSETS,2111,40000.0,u,41,False,False,True,VOLAILLES


In [53]:
farms_stocks = farms_stocks.drop_duplicates(subset="name")

In [54]:
result = farms_stocks.groupby(["departement", "type"])["name"].count().unstack().reset_index()

In [55]:
result[["departement", "BOVINS", "PORCS", "VOLAILLES"]]

type,departement,BOVINS,PORCS,VOLAILLES
0,10,5.0,3.0,12.0
1,11,1.0,9.0,3.0
2,12,16.0,99.0,1.0
3,13,NaN,5.0,1.0
4,14,48.0,45.0,1.0
...,...,...,...,...
76,93,NaN,NaN,1.0
77,971,NaN,6.0,1.0
78,972,NaN,3.0,NaN
79,973,NaN,2.0,NaN


In [56]:
def dominant_type(row) -> str:
    num_bovins = row["BOVINS"] if row["BOVINS"] != np.nan else 0
    num_porcs = row["PORCS"] if row["PORCS"] != np.nan else 0
    num_volailles = row["VOLAILLES"] if row["VOLAILLES"] != np.nan else 0

    type_max = np.nan
    type_max_num = 0
    if num_bovins > type_max_num:
        type_max = "BOVINS"
        type_max_num = num_bovins
    if num_porcs > type_max_num:
        type_max = "PORCS"
        type_max_num = num_porcs
    if num_volailles > type_max_num:
        type_max = "VOLAILLES"
        type_max_num = num_volailles
    
    return type_max


result["dominant_type"] = result.apply(dominant_type, axis=1)

In [57]:
result.to_csv("3_bis.csv", index=False)